In [1]:
using StaticArrays
using LinearAlgebra
using Combinatorics
sa = StaticArrays
la = LinearAlgebra
cm = Combinatorics 

Combinatorics

In [2]:
SPoint = sa.SVector{2, Float64}
SamePoints = Tuple{SPoint, SPoint, SPoint, SPoint}
function first_in_klein(point::SPoint)::SPoint
    x::Float64 = point[1]
    y::Float64 = point[2]
    if (0 <= x <= 1) & (0 <= y <= 1)
        return point

    elseif -1 <= x < 0 < y <= 1
        return @sa.SVector [x + 2.0, 2.0 - y]

    elseif (-1 <= x < 0) & (-1 <= y <= 0)
        return @sa.SVector [x + 2.0, -y]

    elseif (-1 <= y < 0 <= x <=1)
        return @sa.SVector [x, 2.0 + y]
    end
    throw(DomainError(point, "Make sure that the first and second coordinates are in the intreval [-1, 1]"))
end

function samepoints_in_kelin(point:: SPoint)::SamePoints
    firstpoint:: SPoint = first_in_klein(point)
    x:: Float64 = firstpoint[1]
    y:: Float64 = firstpoint[2]
    second:: SPoint = @sa.SVector [x - 2.0, 2.0 - y]
    theard:: SPoint = @sa.SVector [x - 2.0, -y]
    fourth:: SPoint =  @sa.SVector [x, y - 2.0]
    return (firstpoint, second, theard, fourth)
end


samepoints_in_kelin (generic function with 1 method)

In [3]:
point = @sa.SVector [0.5, -1.0]
println(samepoints_in_kelin(point))

([0.5, 1.0], [-1.5, 1.0], [-1.5, -1.0], [0.5, -1.0])


In [4]:
Quadrants = Tuple{Int64, Int64}

function quadrants_square_distance(quadrants:: Quadrants, samepoints1:: SamePoints, samepoints2:: SamePoints)::Float64
    index1:: Int64 = quadrants[1]
    index2:: Int64 = quadrants[2]
    diff:: SPoint = samepoints1[index1]- samepoints2[index2]
    return la.norm(diff)
end

function candidates_quadrants(point1:: SPoint, point2:: SPoint)::Tuple{Quadrants, Quadrants}
    add_second:: Float64 = (point1 + point2)[2]
    diff:: SPoint = point1 - point2
    #print(diff[2], 'in candidate_quadrants')
    #candidate1:: Quadrants = (1, 3) if (add_second < 1.0)  else (1, 2) if (add_second < 3.0) else (4, 2)
    candidate1:: Quadrants = add_second < 1.0 ? (1, 3) : add_second < 3.0 ? (1, 2) : (4, 2)
    candidate2:: Quadrants =  diff[2] < -1.0 ? (1, 4) : diff[2] < 1.0 ? (1, 1) : (2, 3)
    return (diff[1] >= 0 ? reverse(candidate1) : candidate1, candidate2)
end


candidates_quadrants (generic function with 1 method)

In [5]:

function distance_in_klein_bottle(point1:: SPoint, point2:: SPoint)::Float64
    samepoints1:: SamePoints = samepoints_in_kelin(point1)
    samepoints2:: SamePoints = samepoints_in_kelin(point2)
    if samepoints1 === samepoints2
        return 0.0
    end
    point1_in_first:: SPoint = samepoints1[1]
    point2_in_first:: SPoint = samepoints2[1]
    candidates:: Tuple{Quadrants, Quadrants} = candidates_quadrants(point1_in_first, point2_in_first)
    #print(candidates, 'in dist klein')
    dist1:: Float64 = quadrants_square_distance(candidates[1], samepoints1, samepoints2)
    dist2:: Float64 = quadrants_square_distance(candidates[2], samepoints1, samepoints2)
    #print(dist1, dist2)
    distance:: Float64 = minimum((dist1, dist2))
end

function distance_map(pointcloud:: Vector{SPoint}):: Vector{Float64}
    combs:: Base.Generator{Combinatorics.Combinations,Combinatorics.var"#reorder#10"{Vector{SPoint}}} = cm.combinations(pointcloud, 2)
    return map(points:: Vector{SPoint}-> distance_in_klein_bottle(points[1], points[2])::Float64, combs)
end


distance_map (generic function with 1 method)

In [16]:
point1 = @sa.SVector [1.0, 0.3]
point2 = @sa.SVector [-1.0, -0.3]
println(distance_in_klein_bottle(point1, point2))

0.0


In [6]:
function uniformpoints_in_square(;range_min:: Float64, range_max:: Float64, sampling:: Int64, step:: Float64 = 0.00001)::Array{Float64,}
    rng = range_min:step:range_max
    dim:: Int64 = 2
    return rand(rng, (sampling, dim))
end

function staticalize(ndarray:: Array{Float64, })::Vector{SPoint}
    rownum ::Int = size(ndarray, 1)
    return [SPoint(ndarray[i,:]) for i = 1:rownum]
end

staticalize (generic function with 1 method)

In [7]:
pointcloud = uniformpoints_in_square(range_min=-1.0, range_max=1.0, sampling=100)
staticpointcloud = staticalize(pointcloud)
println(typeof(staticpointcloud))
println(staticpointcloud[1])


Array{SArray{Tuple{2},Float64,1,2},1}
[0.96011, -0.74466]


In [8]:
@time distance_map(staticpointcloud)


  0.147819 seconds (535.00 k allocations: 24.667 MiB, 4.42% gc time)


4950-element Array{Float64,1}:
 0.6196559963399048
 0.9548542248950882
 0.7102112660328614
 0.43888924821644937
 1.0597017793700265
 0.6683961317961077
 0.29409886670981905
 0.81427673379509
 0.9419565753260605
 0.7539838535406445
 0.9855644346769011
 1.0746002542341035
 0.8101246928714122
 ⋮
 0.7919853700668972
 0.35652806972242734
 0.27373239486768813
 0.15939234172318317
 1.0572543842425057
 0.6670255213258336
 0.11436077736706748
 0.7836143751871836
 0.6531083944491909
 0.8979629268516601
 0.646641757466992
 1.082560044755024

In [9]:
pointcloud = uniformpoints_in_square(range_min=-1.0, range_max=1.0, sampling=1000)
staticpointcloud = staticalize(pointcloud)
staticpointcloud[1]
println(typeof(staticpointcloud))


Array{SArray{Tuple{2},Float64,1,2},1}


In [12]:
@time distance_map(staticpointcloud)

1.409229069385102